In [31]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import TensorDataset
from tensorboardX import SummaryWriter
import time

from preprocess_pytorch import fetch_data
from sklearn.model_selection import train_test_split

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [32]:
NAME = "15000_pytorch_1v0_fullstate"
SAVE_EVERY = 10 # Save every N epochs

INPUT_SIZE = 59
NUM_EPOCHS = 100
BATCH_SIZE = 32
LEARNING_RATE = 0.0005

# Load Data

In [3]:
X, y = fetch_data(['data/15000_fullstate_1v0.log'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_actions, y_train_parameters = y_train[:,0].astype(np.long), y_train[:,1:]
y_test_actions, y_test_parameters = y_test[:,0].astype(np.long), y_test[:,1:]

In [4]:
num_train_data = X_train.shape[0]
num_test_data = X_test.shape[0]

In [7]:
# Dataset
train_dataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train_actions), torch.from_numpy(y_train_parameters))
test_dataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test_actions), torch.from_numpy(y_test_parameters))

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=BATCH_SIZE, 
                                          shuffle=False)

# Train

In [37]:
# DASH TURN TACKLE KICK 2 1 1 2

# Fully connected neural network with one hidden layer
from pytorch_model import NeuralNet
    
def get_test_loss(model, criterion_actions, criterion_parameters, test_loader, num_test_data):
    test_loss_actions = 0
    test_loss_parameters = 0
    with torch.no_grad():
        for X, y_actions, y_parameters in test_loader:
            X = X.to(device)
            y_actions = y_actions.to(device)
            y_parameters = y_parameters.to(device)
            outputs_actions, output_parameters = model(X)
            loss_actions = criterion_actions(outputs_actions, y_actions)            
            relevance_map = y_parameters != y_parameters
            y_parameters[relevance_map] = 0
            output_parameters[relevance_map] = 0
            
            loss_parameters = criterion_parameters(output_parameters, y_parameters) / 2 / 400
            
            test_loss_actions += loss_actions.item()
            test_loss_parameters += loss_parameters.item()
    return test_loss_actions / num_test_data, test_loss_parameters / num_test_data

In [38]:
model = NeuralNet().to(device).double()
# Loss and optimizer
criterion_actions = nn.CrossEntropyLoss(size_average=False)
criterion_parameters = nn.MSELoss(size_average=False) # MSE Sum
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

writer = SummaryWriter() # Tensorboard

In [39]:
start_time = time.time()
time.sleep(5)
x = time.time() - start_time # seconds

In [40]:
# Train the model
total_step = len(train_loader)
update_interval = total_step // 3
start_time = time.time()
for epoch in range(NUM_EPOCHS):
    train_loss_actions = 0
    train_loss_parameters = 0
    for i, (X, y_actions, y_parameters) in enumerate(train_loader):  
        # Move tensors to the configured device
        X = X.to(device)
        y_actions = y_actions.to(device)
        y_parameters = y_parameters.to(device)
        
        N = X.shape[0] # num of data
        
        # Forward pass
        output_actions, output_parameters = model(X)
        loss_actions = criterion_actions(output_actions, y_actions)
        
        # Figure out relevance using nans
        relevance_map = y_parameters != y_parameters
        y_parameters[relevance_map] = 0
        output_parameters[relevance_map] = 0
        
        loss_parameters = criterion_parameters(output_parameters, y_parameters) / 2 / 400
        loss = (loss_actions + loss_parameters) / N
        
        train_loss_actions += loss_actions
        train_loss_parameters += loss_parameters
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % update_interval == 0:
            print('Epoch [{}/{}], Iteration [{}/{}], Time Elapsed: {:.4f} s'.format(
                epoch+1, NUM_EPOCHS, i + 1, total_step, time.time() - start_time))
        
    train_loss_actions /= num_train_data
    train_loss_parameters /= num_train_data
    train_loss_total = train_loss_actions + train_loss_parameters
    
    test_loss_actions, test_loss_parameters = get_test_loss(
        model, criterion_actions, criterion_parameters, test_loader, num_test_data)
    test_loss_total = test_loss_actions + test_loss_parameters
    
    writer.add_scalar('total_loss/1:training_loss', train_loss_total, epoch + 1)
    writer.add_scalar('total_loss/2:test_loss', test_loss_total, epoch + 1)
    
    writer.add_scalar('action_loss/1:training_loss', train_loss_actions, epoch + 1)
    writer.add_scalar('action_loss/2:test_loss', test_loss_actions, epoch + 1)
    
    writer.add_scalar('parameters_loss/1:training_loss', train_loss_parameters, epoch + 1)
    writer.add_scalar('parameters_loss/2:test_loss', test_loss_parameters, epoch + 1)
    
    if (epoch + 1) % SAVE_EVERY == 0:
        filename = './models/%s_%d.model' % (NAME, epoch)
        torch.save(model.state_dict(), filename)
        print("File %s created" % filename)
    
    print('Epoch [{}/{}], Loss: {:.4f} ({:.4f} + {:.4f}), Test Loss: {:.4f} ({:.4f} + {:.4f})'.format(
        epoch+1, NUM_EPOCHS, 
        train_loss_total, train_loss_actions, train_loss_parameters,
        test_loss_total, test_loss_actions, test_loss_parameters))
    print("Time Elapsed: {:.4f} mins".format((time.time() - start_time) / 60.0))
    
filename = './models/%s_FINAL.model' % (NAME)
torch.save(model.state_dict(), filename)
print("File %s created" % filename)

Epoch [1/100], Iteration [8610/25831], Time Elapsed: 28.3020 s
Epoch [1/100], Iteration [17220/25831], Time Elapsed: 55.9391 s
Epoch [1/100], Iteration [25830/25831], Time Elapsed: 83.5391 s
Epoch [1/100], Loss: 0.2612 (0.1153 + 0.1459), Test Loss: 0.1742 (0.0828 + 0.0914)
Time Elapsed: 1.5698 mins
Epoch [2/100], Iteration [8610/25831], Time Elapsed: 121.9616 s
Epoch [2/100], Iteration [17220/25831], Time Elapsed: 149.6864 s
Epoch [2/100], Iteration [25830/25831], Time Elapsed: 177.4304 s
Epoch [2/100], Loss: 0.1459 (0.0743 + 0.0716), Test Loss: 0.1296 (0.0644 + 0.0652)
Time Elapsed: 3.1376 mins
Epoch [3/100], Iteration [8610/25831], Time Elapsed: 216.2679 s
Epoch [3/100], Iteration [17220/25831], Time Elapsed: 244.0488 s
Epoch [3/100], Iteration [25830/25831], Time Elapsed: 271.7530 s
Epoch [3/100], Loss: 0.1240 (0.0649 + 0.0592), Test Loss: 0.1727 (0.0790 + 0.0937)
Time Elapsed: 4.7087 mins
Epoch [4/100], Iteration [8610/25831], Time Elapsed: 310.6204 s
Epoch [4/100], Iteration [1722

Epoch [27/100], Iteration [17220/25831], Time Elapsed: 2518.4407 s
Epoch [27/100], Iteration [25830/25831], Time Elapsed: 2546.1515 s
Epoch [27/100], Loss: 0.0677 (0.0347 + 0.0330), Test Loss: 0.0641 (0.0320 + 0.0321)
Time Elapsed: 42.6188 mins
Epoch [28/100], Iteration [8610/25831], Time Elapsed: 2585.1160 s
Epoch [28/100], Iteration [17220/25831], Time Elapsed: 2612.9535 s
Epoch [28/100], Iteration [25830/25831], Time Elapsed: 2640.6724 s
Epoch [28/100], Loss: 0.0673 (0.0339 + 0.0334), Test Loss: 0.0901 (0.0462 + 0.0439)
Time Elapsed: 44.1932 mins
Epoch [29/100], Iteration [8610/25831], Time Elapsed: 2679.6928 s
Epoch [29/100], Iteration [17220/25831], Time Elapsed: 2707.5241 s
Epoch [29/100], Iteration [25830/25831], Time Elapsed: 2735.4340 s
Epoch [29/100], Loss: 0.0725 (0.0339 + 0.0387), Test Loss: 0.0758 (0.0389 + 0.0368)
Time Elapsed: 45.7822 mins
Epoch [30/100], Iteration [8610/25831], Time Elapsed: 2775.1265 s
Epoch [30/100], Iteration [17220/25831], Time Elapsed: 2803.0434 s


Epoch [53/100], Iteration [8610/25831], Time Elapsed: 4963.9604 s
Epoch [53/100], Iteration [17220/25831], Time Elapsed: 4991.7611 s
Epoch [53/100], Iteration [25830/25831], Time Elapsed: 5019.5402 s
Epoch [53/100], Loss: 0.0578 (0.0290 + 0.0288), Test Loss: 0.0587 (0.0282 + 0.0305)
Time Elapsed: 83.8440 mins
Epoch [54/100], Iteration [8610/25831], Time Elapsed: 5058.7964 s
Epoch [54/100], Iteration [17220/25831], Time Elapsed: 5086.4426 s
Epoch [54/100], Iteration [25830/25831], Time Elapsed: 5114.1854 s
Epoch [54/100], Loss: 0.0569 (0.0290 + 0.0279), Test Loss: 0.0666 (0.0324 + 0.0343)
Time Elapsed: 85.4249 mins
Epoch [55/100], Iteration [8610/25831], Time Elapsed: 5153.6652 s
Epoch [55/100], Iteration [17220/25831], Time Elapsed: 5181.4270 s
Epoch [55/100], Iteration [25830/25831], Time Elapsed: 5209.0606 s
Epoch [55/100], Loss: 0.0560 (0.0289 + 0.0271), Test Loss: 0.0677 (0.0274 + 0.0403)
Time Elapsed: 87.0019 mins
Epoch [56/100], Iteration [8610/25831], Time Elapsed: 5248.3626 s
E

Epoch [79/100], Iteration [8610/25831], Time Elapsed: 7438.6198 s
Epoch [79/100], Iteration [17220/25831], Time Elapsed: 7466.4193 s
Epoch [79/100], Iteration [25830/25831], Time Elapsed: 7494.2020 s
Epoch [79/100], Loss: 0.0559 (0.0261 + 0.0297), Test Loss: 0.0710 (0.0346 + 0.0365)
Time Elapsed: 125.0881 mins
Epoch [80/100], Iteration [8610/25831], Time Elapsed: 7533.5350 s
Epoch [80/100], Iteration [17220/25831], Time Elapsed: 7561.3822 s
Epoch [80/100], Iteration [25830/25831], Time Elapsed: 7589.2062 s
File ./models/15000_pytorch_1v0_fullstate_79.model created
Epoch [80/100], Loss: 0.0553 (0.0268 + 0.0285), Test Loss: 0.0544 (0.0248 + 0.0296)
Time Elapsed: 126.6775 mins
Epoch [81/100], Iteration [8610/25831], Time Elapsed: 7628.9387 s
Epoch [81/100], Iteration [17220/25831], Time Elapsed: 7656.7872 s
Epoch [81/100], Iteration [25830/25831], Time Elapsed: 7684.6129 s
Epoch [81/100], Loss: 0.0530 (0.0259 + 0.0271), Test Loss: 0.0621 (0.0243 + 0.0378)
Time Elapsed: 128.2672 mins
Epoch